In [15]:
import shapely
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import src.base.find_tie_points as ftp
import src.base.create_mask as cm
import src.display.display_images as di

import src.load.load_image as li

import numpy as np

image_id = "CA181632V0113"

img = li.load_image(image_id)

#mask1 = cm.create_mask(img1, use_default_fiducials=True, 
#default_fid_position=750)
mask = cm.create_mask(img, use_default_fiducials=True, default_fid_position=750)


In [20]:
import shapely
import src.base.connect_to_database as ctd
import src.load.load_satellite as lsd
conn = ctd.establish_connection()

sql_string = f"SELECT ST_AsText(footprint_exact) as footprint_exact FROM images_georef WHERE image_id='{image_id}'"
data = ctd.execute_sql(sql_string, conn)
footprint = data.iloc[0]['footprint_exact']
footprint = shapely.from_wkt(footprint)

sat, _ = lsd.load_satellite(footprint.bounds)

print(sat.shape)

(3, 730, 745)


In [21]:
tpd_basic = ftp.TiePointDetector('lightglue', matching_additional=True, matching_extra=True, display=False, catch=False)

tps, conf = tpd_basic.find_tie_points(sat, img, mask1=None, mask2=mask)

print(tps.shape)

di.display_images([sat, img], tie_points=tps, tie_points_conf=conf)

(107, 4)


In [ ]:
import src.base.resize_image as ri
img1_resized = li.load_image("/data/ATM/papers/georef_paper/revision/resized_img1.tif")
img2_resized = li.load_image("/data/ATM/papers/georef_paper/revision/resized_img2.tif")
mask1_resized = ri.resize_image(mask1, img1_resized.shape)
mask2_resized = ri.resize_image(mask2, img2_resized.shape)

tps_resized = np.load("/data/ATM/papers/georef_paper/revision/resized_tps.npy")
conf_resized = np.load("/data/ATM/papers/georef_paper/revision/resized_conf.npy")
mask_indices_resized = np.load("/data/ATM/papers/georef_paper/revision/resized_mask_indices.npy")

conf_masked_resized = np.array(mask_indices_resized, dtype=int)

style_config = {'axis_marker': False,
                'line_width': 0.4,
                'overlay_alpha': 0.3,
                }

print(tps_resized.shape)
print(np.sum(mask_indices_resized))

di.display_images([img1_resized, img2_resized], 
                  overlays=[mask1_resized, mask2_resized], tie_points=tps_resized, tie_points_conf=conf_masked_resized, style_config=style_config)


In [ ]:
tpd_additional = ftp.TiePointDetector('lightglue', matching_additional=True, matching_extra=False, keep_resized_points=False, display=False, catch=False)

_, _ = tpd_additional.find_tie_points(img1, img2, mask1=mask1, mask2=mask2)


In [ ]:
from shapely.geometry import Polygon


tps_initial = np.load("/data/ATM/papers/georef_paper/revision/initial_tps.npy")
conf_initial = np.load("/data/ATM/papers/georef_paper/revision/initial_conf.npy")

img1_additional = li.load_image("/data/ATM/papers/georef_paper/revision/additional_img1.tif")
img2_additional = li.load_image("/data/ATM/papers/georef_paper/revision/additional_img2.tif")

print(img1_additional.shape)
print(img2_additional.shape)

tps_additional = np.load("/data/ATM/papers/georef_paper/revision/additional_tps.npy")
conf_additional = np.load("/data/ATM/papers/georef_paper/revision/additional_conf.npy")
#mask_indices_additional = np.load("/data/ATM/papers/georef_paper/revision/additional_mask_indices.npy")

print(tps_additional.shape)

tiles1_additional = np.load("/data/ATM/papers/georef_paper/revision/additional_tiles1.npy")
tiles2_additional = np.load("/data/ATM/papers/georef_paper/revision/additional_tiles2.npy")

# Convert bounding boxes to a list of Shapely Polygons
polygons1_additional = []
polygons_color = []
for i, bbox in enumerate(tiles1_additional):
    min_x, min_y, max_x, max_y = bbox
    polygon = Polygon([(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)])
    polygons1_additional.append(polygon)
polygons2_additional = []
for bbox in tiles2_additional:
    min_x, min_y, max_x, max_y = bbox
    polygon = Polygon([(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)])
    polygons2_additional.append(polygon)
    if min_x == 0 and max_x == 0:
        polygons_color.append((255,0,0))
    else:
        polygons_color.append((0,255,0))

def generate_uppercase_sequence(n):    
    def next_sequence(s):
        if s == "":
            return "A"
        if s[-1] != "Z":
            return s[:-1] + chr(ord(s[-1]) + 1)
        return next_sequence(s[:-1]) + "A"
    
    sequence = []
    current = ""
    for _ in range(n):
        current = next_sequence(current)
        sequence.append(current)
    
    return sequence

poly_text1 = generate_uppercase_sequence(len(polygons1_additional))
poly_text2 = generate_uppercase_sequence(len(polygons2_additional))

#conf_masked_additional = np.array(mask_indices_additional, dtype=int)

style_config = {'axis_marker': False,
                'line_width': 0.05,
                'line_color': (0, 255, 0)
                }

di.display_images([img1_additional, img2_additional], 
                  tie_points=tps_additional,
                  #tie_points_conf=conf_additional,
                  polygons=[polygons1_additional, polygons2_additional],polygons_text=[poly_text1, poly_text2],
                  polygons_color=polygons_color,
                  style_config=style_config)


In [ ]:
tpd_extra = ftp.TiePointDetector('lightglue', matching_additional=True, matching_extra=True, keep_resized_points=False, display=False, catch=False)

_, _ = tpd_extra.find_tie_points(img1, img2, mask1=mask1, mask2=mask2)

In [ ]:
from shapely.geometry import Polygon


tps_initial = np.load("/data/ATM/papers/georef_paper/revision/initial_tps.npy")
conf_initial = np.load("/data/ATM/papers/georef_paper/revision/initial_conf.npy")

img1_extra = li.load_image("/data/ATM/papers/georef_paper/revision/extra_img1.tif")
img2_extra = li.load_image("/data/ATM/papers/georef_paper/revision/extra_img2.tif")

tps_extra = np.load("/data/ATM/papers/georef_paper/revision/extra_tps.npy")
conf_extra = np.load("/data/ATM/papers/georef_paper/revision/extra_conf.npy")
#mask_indices_extra = np.load("/data/ATM/papers/georef_paper/revision/extra_mask_indices.npy")

print(tps_extra.shape)

tiles1_extra = np.load("/data/ATM/papers/georef_paper/revision/extra_tiles1.npy")
tiles2_extra = np.load("/data/ATM/papers/georef_paper/revision/extra_tiles2.npy")

# Convert bounding boxes to a list of Shapely Polygons
polygons1_extra = []
polygons_color = []
for i, bbox in enumerate(tiles1_extra):
    min_x, min_y, max_x, max_y = bbox
    polygon = Polygon([(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)])
    polygons1_extra.append(polygon)
polygons2_extra = []
for idx, bbox in enumerate(tiles2_extra):
    min_x, min_y, max_x, max_y = bbox
    polygon = Polygon([(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)])
    polygons2_extra.append(polygon)
    
    if idx in [28, 29]:
        polygons_color.append((255,0,0))
        polygons2_extra[idx] = Polygon([(0,0), (0,0), (0,0), (0,0)])
    elif (min_x == 0 and max_x == 0):
        polygons_color.append((255,0,0))
    else:
        polygons_color.append((0,255,0))

def generate_uppercase_sequence(n):    
    def next_sequence(s):
        if s == "":
            return "A"
        if s[-1] != "Z":
            return s[:-1] + chr(ord(s[-1]) + 1)
        return next_sequence(s[:-1]) + "A"
    
    sequence = []
    current = ""
    for _ in range(n):
        current = next_sequence(current)
        sequence.append(current)
    
    return sequence

poly_text1 = generate_uppercase_sequence(len(polygons1_extra))
poly_text2 = generate_uppercase_sequence(len(polygons2_extra))

#conf_masked_additional = np.array(mask_indices_additional, dtype=int)

style_config = {'axis_marker': False,
                'line_width': 0.05,
                'line_color': (0, 255, 0)
                }

di.display_images([img1_extra, img2_extra], 
                  #tie_points=tps_extra,
                  #tie_points_conf=conf_extra,
                  polygons=[polygons1_extra, polygons2_extra],polygons_text=[poly_text1, poly_text2],
                  polygons_color=polygons_color,
                  style_config=style_config)
